In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import pickle

import pybamm
from datetime import datetime, timedelta
from time import time
import mscc_aging_test_DOD100 as worker
import multiprocessing as mp
from scipy.optimize import minimize
from joblib import Parallel, delayed

from LIB_GP import*
from LIB_acq_fun_min import*

print(pybamm.__version__)
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 12

24.9.0


In [2]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore", ConvergenceWarning)

In [3]:
set_T_C = 25 
n_MV = 5
# cap_spec = 0.80 # capacity retention spec

In [4]:
n_opt_start = 100

### 1. Initial data generation

In [5]:
ecm_data = np.load('ecm_data_var5.npy')
ecm_data_X = ecm_data[:,0:n_MV]
ecm_data_y = ecm_data[:,n_MV:]

In [6]:
print(ecm_data_X)
print(ecm_data_y)

[[0.2  0.2  0.2  3.7  3.95]
 [0.2  0.2  0.2  3.7  4.  ]
 [0.2  0.2  0.2  3.7  4.05]
 ...
 [2.   2.   1.   3.85 4.  ]
 [2.   2.   1.   3.85 4.05]
 [2.   2.   1.   3.85 4.1 ]]
[[4.99978431 0.86508649]
 [4.99978436 0.86508665]
 [4.99978443 0.86508783]
 ...
 [0.66799437 0.72775597]
 [0.64014206 0.72045853]
 [0.61476164 0.71379608]]


In [7]:
len(ecm_data)

14400

In [8]:
import joblib

# 저장된 모델 불러오기
ecm_interp_knn = joblib.load('ecm_interp_knn.pkl')

# 불러온 모델로 예측
y_pred = ecm_interp_knn.predict(np.array([2.0,0.2,2.0,3.7,4.05]).reshape(-1,n_MV))
print(y_pred)

[[3.83060566 0.82566393]]


In [9]:
spec_candi = [0.86,0.84,0.82,0.80]

In [10]:
X_best = []
for i,spec in enumerate(spec_candi):
    print(spec)
    opt_org = np.argmin(ecm_data_y[:,0][ecm_data_y[:,1]>=spec])
    X_best_temp = ecm_data_X[ecm_data_y[:,1]>=spec][opt_org]
    
    X_best.append(X_best_temp)

0.86
0.84
0.82
0.8


In [11]:
X_best = np.array(X_best).reshape(-1,5)
X_best

array([[0.2 , 0.4 , 0.3 , 3.7 , 4.05],
       [0.8 , 0.6 , 1.  , 3.7 , 4.1 ],
       [1.  , 1.2 , 1.  , 3.8 , 3.95],
       [1.4 , 1.4 , 1.  , 3.7 , 4.05]])

In [12]:
ecm_interp_knn.predict(X_best)

array([[3.64567067, 0.86012611],
       [1.49729116, 0.84020466],
       [0.97002286, 0.82002044],
       [0.78542323, 0.80037281]])

In [13]:
X_initial = np.array(X_best).flatten().reshape(-1,5)

In [14]:
X_initial

array([[0.2 , 0.4 , 0.3 , 3.7 , 4.05],
       [0.8 , 0.6 , 1.  , 3.7 , 4.1 ],
       [1.  , 1.2 , 1.  , 3.8 , 3.95],
       [1.4 , 1.4 , 1.  , 3.7 , 4.05]])

In [15]:
n_initial = X_initial.shape[0]

In [16]:
num_core = mp.cpu_count()
num_core = 4
print(num_core)
p = mp.Pool(processes = num_core)
st_time = time()
initial_data = p.starmap(worker.aging_test_results,X_initial)  # x_exp, y_exp, avg_temp
ed_time = time()
print('Time',np.round((ed_time-st_time),2)/60, 'minutes')
p.close()
p.join()

4
Time 9.6085 minutes


In [17]:
initial_data

[[3.6358700431704136, 0.8616513715236432],
 [1.4930488550570296, 0.8476182640206333],
 [0.9669532970602539, 0.8302684806988],
 [0.7826535327190598, 0.8150948601186301]]

In [18]:
initial_data = np.array(initial_data)

In [19]:
data_save = np.concatenate([X_initial,initial_data],axis=1)

In [20]:
np.save('PrBO_initial_data.npy',data_save)

In [21]:
np.load('PrBO_initial_data.npy')

array([[0.2       , 0.4       , 0.3       , 3.7       , 4.05      ,
        3.63587004, 0.86165137],
       [0.8       , 0.6       , 1.        , 3.7       , 4.1       ,
        1.49304886, 0.84761826],
       [1.        , 1.2       , 1.        , 3.8       , 3.95      ,
        0.9669533 , 0.83026848],
       [1.4       , 1.4       , 1.        , 3.7       , 4.05      ,
        0.78265353, 0.81509486]])

In [22]:
data_save[:,-2]*60

array([218.15220259,  89.5829313 ,  58.01719782,  46.95921196])

In [23]:
import numpy as np
import pandas as pd
import joblib
# 저장된 모델 불러오기
ecm_interp_knn = joblib.load('ecm_interp_knn.pkl')
data_save = np.load('PrBO_initial_data.npy')
ecm_initial = ecm_interp_knn.predict(data_save[:,:5])


In [24]:
data_save

array([[0.2       , 0.4       , 0.3       , 3.7       , 4.05      ,
        3.63587004, 0.86165137],
       [0.8       , 0.6       , 1.        , 3.7       , 4.1       ,
        1.49304886, 0.84761826],
       [1.        , 1.2       , 1.        , 3.8       , 3.95      ,
        0.9669533 , 0.83026848],
       [1.4       , 1.4       , 1.        , 3.7       , 4.05      ,
        0.78265353, 0.81509486]])

In [25]:
data_concat = np.concatenate([data_save[:,:5],ecm_initial,data_save[:,5:]],axis=1)

In [26]:
data_concat

array([[0.2       , 0.4       , 0.3       , 3.7       , 4.05      ,
        3.64567067, 0.86012611, 3.63587004, 0.86165137],
       [0.8       , 0.6       , 1.        , 3.7       , 4.1       ,
        1.49729116, 0.84020466, 1.49304886, 0.84761826],
       [1.        , 1.2       , 1.        , 3.8       , 3.95      ,
        0.97002286, 0.82002044, 0.9669533 , 0.83026848],
       [1.4       , 1.4       , 1.        , 3.7       , 4.05      ,
        0.78542323, 0.80037281, 0.78265353, 0.81509486]])

In [27]:
df_save = pd.DataFrame(data_concat)
df_save.columns= ['I1','I2','I3','V1','V2','ECM tc [min]','ECM Cr [%]', 'EM tc [min]','EM Cr [%]']
df_save['ECM tc [min]'] = df_save['ECM tc [min]']*60
df_save['ECM Cr [%]'] = df_save['ECM Cr [%]']*100

df_save['EM tc [min]'] = df_save['EM tc [min]']*60
df_save['EM Cr [%]'] = df_save['EM Cr [%]']*100

df_save.to_csv('PrBO_initial_samples.csv',index=False)
df_save

,I1,I2,I3,V1,V2,ECM tc [min],ECM Cr [%],EM tc [min],EM Cr [%]
0,0.2,0.4,0.3,3.7,4.05,218.740240,86.012611,218.152203,86.165137
1,0.8,0.6,1.0,3.7,4.10,89.837470,84.020466,89.582931,84.761826
2,1.0,1.2,1.0,3.8,3.95,58.201372,82.002044,58.017198,83.026848
3,1.4,1.4,1.0,3.7,4.05,47.125394,80.037281,46.959212,81.509486


In [28]:
np.load('PrBO_initial_data.npy')

array([[0.2       , 0.4       , 0.3       , 3.7       , 4.05      ,
        3.63587004, 0.86165137],
       [0.8       , 0.6       , 1.        , 3.7       , 4.1       ,
        1.49304886, 0.84761826],
       [1.        , 1.2       , 1.        , 3.8       , 3.95      ,
        0.9669533 , 0.83026848],
       [1.4       , 1.4       , 1.        , 3.7       , 4.05      ,
        0.78265353, 0.81509486]])